In [1]:
from context import jellecomplete

In [2]:
import jellecomplete.utils as utils
print(utils.hello_world())

hello world!


In [3]:
# import a mesh
import jellecomplete.complete as complete
import open3d as o3d

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/pointcloud-xr-completion/samples/BigFiles/GrondSampleMesh.obj"
mesh = utils.get_geometry(meshPath)

In [4]:
# import pointcloud
pcdPath = "/Volumes/Data drive/Documents/Doctoraat Local/pointcloud-xr-completion/samples/BigFiles/voxel_grond_pointcloud.ply"
pcd = utils.get_geometry(pcdPath)

In [5]:
utils.show_geometries([mesh, pcd])

In [5]:
pcl = mesh.sample_points_poisson_disk(number_of_points=10000)
hull, _ = pcl.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([mesh, hull_ls])

In [6]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [7]:
import numpy as np
hullVerts = np.asarray(hull.vertices)
points = np.asarray(pcd.points)
#print(hullVerts)

In [8]:

ind = in_hull(points, hullVerts)
intList = []
for i,x in enumerate(ind):
    if ind[i]:
        intList.append(i)


In [17]:

pcd_in_hull = pcd.select_by_index(intList)
print(len(pcd_in_hull.points))
utils.show_geometries([pcd_in_hull, pcl])

33161


In [21]:
#distance filtering the original point cloud
dists = pcd_in_hull.compute_point_cloud_distance(pcl)
dists = np.asarray(dists)
ind = np.where(dists < 0.05)[0]
pcd_without_mesh = pcd_in_hull.select_by_index(ind)
utils.show_geometries([pcd_without_mesh,pcl])

In [23]:
#distance filtering the new point cloud
dists = pcl.compute_point_cloud_distance(pcd_in_hull)
dists = np.asarray(dists)
ind = np.where(dists > 0.05)[0]
pcl_new = pcl.select_by_index(ind)
utils.show_geometries([pcl_new,pcd_without_mesh])

In [24]:
#combining the 2 pointclouds
combined_pcd = pcl_new + pcd_without_mesh

utils.show_geometries([combined_pcd])

In [25]:
pcd_out_hull = pcd.select_by_index(intList, True)

finalPcd = pcd_out_hull + combined_pcd

utils.show_geometries([finalPcd])